In [ ]:
from bs4 import BeautifulSoup
from lib.import_lang_data import *
from lib.lang_featurizers import *
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# read in assignment's test files
X_assignment_test, y_assignment_test = read_tests()

# read corpus into lang_data, lang_results
languages = ['gcc', 'c', 'csharp', 'sbcl', 'clojure', 'ghc' 'java', 'javascript',
             'ocaml', 'perl', 'php', 'hack', 'py', 'python3', 'jruby', 'yarv', 'rb',
             'scala', 'racket']
lang_data, lang_results = read_polyglot(languages)
lang_info = pd.DataFrame(lang_results)
lang_info[0].value_counts()

In [ ]:
lang_info = match_extensions(lang_info)
lang_results = list(lang_info[0])
lang_info[0].value_counts()

In [ ]:
lang_featurizer = make_union(
    BagOfWordsFeaturizer(52),
    FunctionFeaturizer(presence_nil,
                       presence_nil_caps,
                       presence_null,
                       presence_none,
#                        presence_start_double_semicolons,
#                        presence_start_hashes,
#                        presence_bar_hash,
                       presence_paren_define,
                       percent_start_and_end_parenthesis,
                       longest_run_of_parenthesis,
                       longest_run_of_curly_braces,
                       single_closing_braces_per_line,
                       presence_function_js,
                       presence_while,
                       presence_do,
                       presence_var,
                       presence_for_js,
#                        final_semicolons_per_line,
                       presence_void,
                       presence_public,
                       presence_bool,
                       presence_int,
                       presence_module_line,
                       presence_extend_line,
                       presence_require_line,
                       presence_end,
                       presence_multiple_end,
                       presence_def_no_colon,
                       presence_at,
                       presence_double_at,
                       presence_puts,
                       presence_dot_times,
                       presence_paren_defn,
                       presence_paren_ns,
                       percent_consecutive_closing_paren,
                       presence_taskloop,
                       presence_runtask,
                       presence_from_import_line,
                       presence_import_line,
                       presence_print_paren,
                       presence_dot_join,
                       presence_dot_format,
                       presence_dot_values,
                       presence_dunder_name,
                       presence_dunder_init,
                       presence_def_colon,
                       
                       )
)

X_train, X_test, y_train, y_test = train_test_split(lang_data, lang_results)

pipe = make_pipeline(lang_featurizer, DecisionTreeClassifier())
pipe.fit(X_train, y_train)
print('R^2 score: {}\n'.format(pipe.score(X_test, y_test)))
print(classification_report(y_test, pipe.predict(X_test)))

## Now to test with the assignment's tests

In [ ]:
print('R^2 score: {}\n'.format(pipe.score(X_assignment_test, y_assignment_test)))
print(classification_report(y_assignment_test, pipe.predict(X_assignment_test)))
print(confusion_matrix(y_assignment_test, pipe.predict(X_assignment_test)))